In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import pandas as pd
from scipy.stats import weibull_min
import scipy
from SALib.sample import saltelli
from SALib.analyze import sobol



%run 'methods.ipynb'

%matplotlib inline

In [2]:
class EnergyModel(object):
    
    def __init__(self, name=None, t=None, i=None, s_c=None,r_s_c=None, r_i=None,
                 data_start_year=None,
                 size=None, power=None, usage=None):
        
        """Init function. Assign the input data to the instance of the object."""
        
        self.name = name
        self.t = t 
        self.i = i  
        self.s_c = s_c
        self.r_s_c = r_s_c
        self.r_i = r_i
        
        self.data_start_year = data_start_year
        
        self.size = size
        self.power = power
        self.usage = usage
        
    
    def get_embodied_energy(self, device, size, year):

        switcher = {
            "WashingMachine": 16.9*((6.4*size)+33.2) if year < 2012 else 16.9*((5.1*size)+31.4),

            "FridgeFreezer": 5.4*(0.19*size),

            "TV": 1014 * np.exp(0.0256 * size),

            "Monitor": 1014 * np.exp(0.0256 * size),
            
            "Desktop": 132.54 * size,

            "Laptop": 132.54 * size,

            "Smartphone": 525 * size,

           }

        return switcher.get(device, "Invalid Device")
    
    def get_operational_energy(self, device, size, power, usage):

        switcher = {
            "WashingMachine": (size/3.3) * power * usage,

            "FridgeFreezer": (size*power),

            "TV": (((size**2) *0.427) * power * usage * 365.5) / 1000,

            "Monitor": (((size**2) *0.427) * power * usage * 365.5) / 1000 ,
            
            "Desktop": (power * usage * 365.5) / 1000,

            "Laptop": (power * usage * 365.5) / 1000 ,

            "Smartphone": (power * usage * 365.5) / 1000 ,

           }

        return switcher.get(device, "Invalid Device")        
        
        
    def get_repair_energy(self, device, size, year):

        switcher = {
            "WashingMachine": 0.26*16.9*((6.4*size)+33.2) if year < 2012 else 0.26*16.9*((5.1*size)+31.4),

            "FridgeFreezer": 0.08*5.4*(0.19*size) ,

            "TV": 0.43*(1014*np.exp(0.026 * size)) ,

            "Monitor": 0.43*(1014*np.exp(0.026 * size)) ,
            
            "Desktop": 0.129 * (132.5 * size) ,

            "Laptop":  0.163 * (132.5 * size)  ,

            "Smartphone": 0.251 * (525 * size) ,

           }

        return switcher.get(device, "Invalid Device")    
     
    
    def calculate_embodied_energy(self):
        
        annual_embodied_energy = []
        year = self.data_start_year
            
        for index, sales in enumerate(self.i):
            
            annual_embodied_energy.append(sales*self.get_embodied_energy(device=self.name, 
                                                                         size=self.size[index],
                                                                         year=year))
            
            year = year+1
        
        
        return annual_embodied_energy
    
    
    def calculate_operational_energy(self):
        
        primary_op_energy = np.zeros((len(self.t), len(self.t)))
        secondary_op_energy = np.zeros((len(self.t), len(self.t)))
        annual_op_energy = np.zeros(len(self.t))
        
        for m in range(0, len(self.t)):
            for n in range(0, len(self.t)):                    
                    
                primary_op_energy[m,n] = self.s_c[m,n]*self.get_operational_energy(device=self.name, 
                                                                                   size=self.size[n], 
                                                                                   power=self.power[n], 
                                                                                   usage=self.usage[n])  
                
                secondary_op_energy[m,n] = self.r_s_c[m,n] * self.get_operational_energy(device=self.name,
                                                                                         size=self.size[n],
                                                                                         power=self.power[n],
                                                                                         usage=self.usage[n])     
        
        primary = pd.DataFrame(data = primary_op_energy)
        secondary = pd.DataFrame(data = secondary_op_energy)
        
        primary.fillna(0)
        secondary.fillna(0)
        
        for m in range (0, len(self.t)):

            annual_op_energy[m] = primary.iloc[m,:].sum() + secondary.iloc[m,:].sum()
        
        return annual_op_energy
    
    def calculate_repair_energy(self):

        repair_energy_cohort = np.zeros((len(self.t), len(self.t)))
        annual_repair_energy = np.zeros(len(self.t))
        
        
        year = self.data_start_year
        
        for m in range(0, len(self.t)):
            for n in range(0, len(self.t)):
                
                repair_energy_cohort[m,n] = self.r_i[m,n]*self.get_repair_energy(device=self.name,
                                                                                 size=self.size[n],
                                                                                 year=year)
            year=year+1
        
        annual_repair_df = pd.DataFrame(data=repair_energy_cohort)
        
        annual_repair_df.dropna(axis=0)
        
        for m in range(len(self.t)):
        
            annual_repair_energy[m] = annual_repair_df.iloc[m,:].sum()
        
        return annual_repair_energy
            
    
    def calculate_energies(self):
        
        em_energy = self.calculate_embodied_energy()
        op_energy = self.calculate_operational_energy()
        re_energy = self.calculate_repair_energy()

        total_energy = em_energy+op_energy+re_energy
        
        return total_energy
    
    def calculate_final_energy(self):

        em_energy = self.calculate_embodied_energy()
        op_energy = self.calculate_operational_energy()
        re_energy = self.calculate_repair_energy()
        
        return em_energy[-1]+op_energy[-1]+re_energy[-1]
    
    def calculate_cumulative_energy(self):

        em_energy = self.calculate_embodied_energy()
        op_energy = self.calculate_operational_energy()
        re_energy = self.calculate_repair_energy()
        
        return em_energy+op_energy+re_energy
